In [11]:
# --- Imports ---
import os
import glob
import re
from datetime import datetime
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
from PIL import Image, ImageDraw, ImageFont
import imageio
import warnings
import csv
import platform # To help find system fonts

# --- Add Geopandas and Rasterio features ---
try:
    import geopandas as gpd
    from rasterio.features import rasterize
    from rasterio.transform import from_bounds
    GEOPANDAS_AVAILABLE = True
except ImportError:
    GEOPANDAS_AVAILABLE = False
    print("WARNING: Geopandas not found. Country masking feature will be disabled.")


# --- Helper function to parse date from filename ---
def get_date_from_filename(filename):
    match = re.search(r'_(\d{4})_(\d{2})\.tif$', os.path.basename(filename))
    if match: year, month = map(int, match.groups()); return datetime(year, month, 1)
    else: print(f"Warning: Could not parse date from filename: {filename}. Skipping."); return None

# --- Main Function ---
def create_nightlight_timelapse_and_graph(
    input_folder: str,
    output_path_base: str,
    output_format: str = 'gif',
    cmap_name: str = 'plasma',
    fps: int = 6,
    normalize_animation: bool = False,
    mask_outside_country: bool = True,
    country_boundary_shapefile_path: str = None,
    target_country_name: str = None,
    outside_mask_color: tuple = (0, 0, 0, 0),
    mp4_crf: int = 23,
    text_size_category: str = 'medium',
    font_path: str = None,
    graph_title: str = 'Average Night Light Intensity Over Time',
    watermark_text: str = 'My Custom Watermark',
    watermark_position: tuple = (10, 10),
    debug_masking: bool = False
    ):
    """ Docstring """

    print(f"Starting analysis for folder: {input_folder}")
    print(f"Output base: {output_path_base}")
    print(f"Format: {output_format}, FPS: {fps}, Colormap: {cmap_name}")
    print(f"Normalize Animation Frames: {normalize_animation}")
    print(f"Mask Outside Country: {mask_outside_country}")
    if mask_outside_country: print(f"Target Country Name: '{target_country_name}'")
    if output_format.lower() == 'mp4': print(f"MP4 CRF Value: {mp4_crf}")
    print(f"Text Size Category: {text_size_category}")
    if debug_masking: print("!!! Mask Debugging Enabled !!!")

    # --- Validate Inputs ---
    if mask_outside_country and not GEOPANDAS_AVAILABLE: print("WARNING: Geopandas not installed. Disabling country masking."); mask_outside_country = False
    if mask_outside_country and not country_boundary_shapefile_path: print("WARNING: 'country_boundary_shapefile_path' not provided. Disabling country masking."); mask_outside_country = False
    if mask_outside_country and not target_country_name: print("WARNING: 'target_country_name' not provided. Disabling country masking."); mask_outside_country = False
    if mask_outside_country and not os.path.exists(country_boundary_shapefile_path): print(f"WARNING: Country shapefile not found at '{country_boundary_shapefile_path}'. Disabling country masking."); mask_outside_country = False


    # --- 1. Find and Sort TIFF Files ---
    search_pattern = os.path.join(input_folder, '*.tif'); tif_files = glob.glob(search_pattern)
    if not tif_files: print(f"Error: No .tif files found in {input_folder}"); return
    file_date_pairs=[(get_date_from_filename(f),f) for f in tif_files]; file_date_pairs=[p for p in file_date_pairs if p[0] is not None]
    if not file_date_pairs: print(f"Error: Could not parse dates"); return
    file_date_pairs.sort(); sorted_files=[f[1] for f in file_date_pairs]; sorted_dates=[f[0] for f in file_date_pairs]; date_labels=[d.strftime('%Y-%m') for d in sorted_dates]
    print(f"Found and sorted {len(sorted_files)} TIFF files.")
    raster_transform = None; raster_shape = None; raster_crs = None; country_mask_array = None; country_mask_available = False
    try:
        with rasterio.open(sorted_files[0]) as src: raster_transform = src.transform; raster_shape = src.shape; raster_crs = src.crs; print(f"Raster Properties: Shape={raster_shape}, CRS={raster_crs}")
    except Exception as e: print(f"ERROR: Could not read metadata: {e}"); return


    # --- Load Font ---
    font = None; calculated_font_size = 10; ttf_font_found = False
    print("\nAttempting to load font...")
    # --- [ FONT LOADING LOGIC - Copied from previous correct version ] ---
    def find_system_font(font_names):
        system = platform.system(); common_paths = []
        if system == "Windows": common_paths = [os.path.join(os.environ.get("SystemRoot", "C:\\Windows"), "Fonts")]
        elif system == "Linux": common_paths = ["/usr/share/fonts/truetype/dejavu", "/usr/share/fonts/truetype/msttcorefonts", "/usr/share/fonts/truetype/liberation", "/usr/share/fonts/truetype", "/usr/local/share/fonts", os.path.expanduser("~/.fonts")]
        elif system == "Darwin": common_paths = ["/System/Library/Fonts", "/Library/Fonts", os.path.expanduser("~/Library/Fonts")]
        for name in font_names:
            try: temp_font = ImageFont.truetype(name, 10); return name
            except IOError: pass
            for directory in common_paths:
                 for ext in ['.ttf', '.otf']:
                      potential_path = os.path.join(directory, name + ext)
                      if os.path.exists(potential_path): return potential_path
                 try:
                     for fname in os.listdir(directory):
                         if name.lower() in fname.lower() and (fname.lower().endswith(".ttf") or fname.lower().endswith(".otf")):
                             potential_path = os.path.join(directory, fname); return potential_path
                 except OSError: continue
        return None
    if font_path and os.path.exists(font_path):
        try: base_size = max(12, int(raster_shape[0]/50)); mult={'small':0.7,'medium':1.0,'large':1.4}.get(text_size_category.lower(),1.0); calculated_font_size = int(base_size*mult); font=ImageFont.truetype(font_path, calculated_font_size); print(f"Loaded specified font: {font_path}"); ttf_font_found = True
        except Exception as e: print(f"Warn: Load specified font fail: {e}")
    if not ttf_font_found:
        found_font_path = find_system_font(['DejaVuSans', 'arial', 'Arial', 'LiberationSans-Regular'])
        if found_font_path:
            try: base_size = max(12, int(raster_shape[0]/50)); mult={'small':0.7,'medium':1.0,'large':1.4}.get(text_size_category.lower(),1.0); calculated_font_size = int(base_size*mult); font=ImageFont.truetype(found_font_path, calculated_font_size); print(f"Loaded system font: {found_font_path}"); ttf_font_found = True
            except Exception as e: print(f"Warn: Load system font fail: {e}")
    if not ttf_font_found:
        try: font=ImageFont.load_default(); print("Warn: Loaded default PIL font"); calculated_font_size=10
        except IOError: print("ERROR: Font load failed"); font = None
    # --- [ END FONT LOADING ] ---


    # --- Prepare Country Mask ---
    if mask_outside_country:
        print(f"\nDEBUG MASK: Preparing country mask for '{target_country_name}'...")
        try:
            world_gdf = gpd.read_file(country_boundary_shapefile_path)
            if debug_masking: print(f"  DEBUG MASK: Loaded shapefile. Columns: {world_gdf.columns.to_list()}")
            possible_name_cols = ['NAME', 'ADMIN', 'SOVEREIGNT', 'name', 'admin', ' sovereignt', 'NAME_0', 'COUNTRY']
            target_geom = None; target_found = False; country_gdf = None
            for col in possible_name_cols:
                 if col in world_gdf.columns:
                      if debug_masking: print(f"  DEBUG MASK: Checking column '{col}' for '{target_country_name}'...")
                      country_gdf_candidate = world_gdf[world_gdf[col] == target_country_name]
                      if not country_gdf_candidate.empty:
                           country_gdf = country_gdf_candidate; target_geom = country_gdf.geometry; print(f"  DEBUG MASK: Found '{target_country_name}' in column '{col}'."); target_found = True; break
            if not target_found: raise ValueError(f"Could not find country '{target_country_name}'")
            if debug_masking: print(f"  DEBUG MASK: Orig CRS: {country_gdf.crs}")

            # --- CORRECTED SYNTAX HERE ---
            if country_gdf.crs != raster_crs:
                print(f"  DEBUG MASK: Reprojecting...")
                country_gdf = country_gdf.to_crs(raster_crs)
                target_geom = country_gdf.geometry
                # Put the optional debug print on a new line
                if debug_masking:
                    print(f"  DEBUG MASK: Reprojected CRS: {country_gdf.crs}")
            # --- END CORRECTION ---

            print("  Rasterizing country polygon...")
            country_mask_array = rasterize(shapes=[(geom, 1) for geom in target_geom], out_shape=raster_shape, transform=raster_transform, fill=0, dtype='uint8')
            country_mask_array = country_mask_array.astype(bool); country_mask_available = True
            if debug_masking: unique, counts = np.unique(country_mask_array, return_counts=True); print(f"  DEBUG MASK: Mask unique values & counts: {dict(zip(unique, counts))}")
            print(f"  Country mask created successfully.")
        except Exception as e:
            print(f"ERROR: Failed during country mask creation: {e}")
            mask_outside_country = False


    # --- 2. Process Files: Calculate Raw Stats & Determine Range ---
    monthly_avg_intensity = []; all_valid_data_list = []; files_with_valid_data_count = 0; max_raw_value_overall = -np.inf
    print("\n--- Pass 1: Calculating Raw Statistics & Preparing Range ---")
    for i, filepath in enumerate(sorted_files):
        try:
            with rasterio.open(filepath) as src:
                raw_data = src.read(1).astype(np.float32)
                with warnings.catch_warnings(): warnings.simplefilter("ignore"); current_max_raw = np.nanmax(raw_data)
                if np.isfinite(current_max_raw) and current_max_raw > max_raw_value_overall: max_raw_value_overall = current_max_raw
                invalid_mask = (np.isnan(raw_data)) | (raw_data <= 0); valid_data = raw_data[~invalid_mask]
                if valid_data.size > 0:
                    files_with_valid_data_count += 1
                    with warnings.catch_warnings(): warnings.simplefilter("ignore"); min_val, max_val, mean_val = np.nanmin(valid_data), np.nanmax(valid_data), np.nanmean(valid_data)
                    if not (np.isfinite(min_val) and np.isfinite(max_val) and np.isfinite(mean_val)): monthly_avg_intensity.append(np.nan)
                    else: monthly_avg_intensity.append(mean_val); all_valid_data_list.append(valid_data)
                else: monthly_avg_intensity.append(np.nan)
        except Exception as e:
            print(f"  ERROR reading {os.path.basename(filepath)}: {e}")
            while len(monthly_avg_intensity) < i + 1: monthly_avg_intensity.append(np.nan)

    # --- [ Pass 1 Summary and Range Determination ] ---
    print(f"\n--- Pass 1 Summary ---"); print(f"  Valid data in {files_with_valid_data_count} files.")
    if not np.isfinite(max_raw_value_overall): max_raw_value_overall = 1.0
    print(f"  Overall max raw value: {max_raw_value_overall:.4f}")
    vis_vmin, vis_vmax = 0.0, max(1.0, max_raw_value_overall)
    if all_valid_data_list:
        print("  Determining vis range via percentiles...");
        try:
            all_valid_data_list_filt=[arr for arr in all_valid_data_list if isinstance(arr,np.ndarray) and arr.size>0]
            if all_valid_data_list_filt:
                 concatenated_data = np.concatenate(all_valid_data_list_filt); del all_valid_data_list
                 if concatenated_data.size > 0:
                     with warnings.catch_warnings(): warnings.simplefilter("ignore"); p_low=np.nanpercentile(concatenated_data,0.5); p_high=np.nanpercentile(concatenated_data,99.5)
                     if np.isfinite(p_low) and np.isfinite(p_high) and p_high>p_low: vis_vmin=p_low; vis_vmax=p_high; print(f" Using 0.5-99.5 range: {vis_vmin:.4f}-{vis_vmax:.4f}")
                     else: print(" Warn: Percentile fail. Fallback.");
                 del concatenated_data
        except Exception as e: print(f"ERROR range calc: {e}")
    if vis_vmax <= vis_vmin: vis_vmax = vis_vmin + 1.0
    if vis_vmin < 0: vis_vmin = 0.0
    if normalize_animation: print(f"--- Using NORMALIZED anim range: {vis_vmin:.4f}-{vis_vmax:.4f} ---"); norm=mcolors.Normalize(vmin=vis_vmin, vmax=vis_vmax)
    else: fixed_vmin, fixed_vmax = 0.0, vis_vmax; print(f"--- Using FIXED anim scaling: {fixed_vmin:.4f}-{fixed_vmax:.4f} ---"); norm=mcolors.Normalize(vmin=fixed_vmin, vmax=fixed_vmax)


    # --- 3. Save CSV ---
    csv_filename = f"{output_path_base}_intensity_data.csv"; print(f"\nSaving CSV: {csv_filename}...")
    try:
        with open(csv_filename, 'w', newline='') as csvf: csvw=csv.writer(csvf); csvw.writerow(['Date','Average_Radiance']); c=0;[csvw.writerow([d.strftime('%Y-%m-%d'), f"{i:.6f}"]) for d,i in zip(sorted_dates, monthly_avg_intensity) if np.isfinite(i) and (c:=c+1)]; print(f" Wrote {c} rows.")
    except Exception as e: print(f"ERROR CSV: {e}")


    # --- 4. Prepare Frames ---
    frames = []; cmap = cm.get_cmap(cmap_name); print("\n--- Pass 2: Creating animation frames ---")
    if len(outside_mask_color) == 4: outside_mask_color_np = np.array(outside_mask_color[:4]) / 255.0
    else: print("Warn: Invalid outside_mask_color"); outside_mask_color_np = np.array([0.0,0.0,0.0,0.0])
    for i, filepath in enumerate(sorted_files):
        if debug_masking: print(f"--- Frame {i+1} Debug ---")
        try:
            with rasterio.open(filepath) as src:
                data = src.read(1).astype(np.float32); nan_mask = (np.isnan(data)) | (data <= 0); data[nan_mask] = np.nan
                data_processed = norm(np.nan_to_num(data, nan=norm.vmin)); rgba_image = cmap(data_processed)
                if mask_outside_country and country_mask_available:
                    if debug_masking: print("  DEBUG MASK Pass 2: Applying country mask...")
                    outside_pixels_mask = ~country_mask_array; num_outside = np.sum(outside_pixels_mask)
                    if debug_masking: print(f"    Pixels targeted OUTSIDE: {num_outside}")
                    if num_outside > 0:
                         alpha_before_min = np.min(rgba_image[outside_pixels_mask, 3]); alpha_before_max = np.max(rgba_image[outside_pixels_mask, 3])
                         if outside_mask_color_np[3] == 0.0: rgba_image[outside_pixels_mask, 3] = 0.0; #if debug_masking: print("    Set OUTSIDE alpha to 0.0")
                         else: rgba_image[outside_pixels_mask, :] = outside_mask_color_np; #if debug_masking: print(f"    Set OUTSIDE color")
                         if debug_masking: alpha_after_min=np.min(rgba_image[outside_pixels_mask, 3]); alpha_after_max=np.max(rgba_image[outside_pixels_mask, 3]); print(f" OUTSIDE Alpha: Before=({alpha_before_min:.2f}-{alpha_before_max:.2f}), After=({alpha_after_min:.2f}-{alpha_after_max:.2f})")
                pil_image = Image.fromarray((rgba_image * 255).astype(np.uint8), 'RGBA'); draw = ImageDraw.Draw(pil_image)
                if font:
                    padding = max(5, int(calculated_font_size*0.5))
                    if watermark_text:
                        try: draw.text((padding,padding), watermark_text, fill=(255,255,255,180), font=font)
                        except Exception as text_err: print(f"Warn: Watermark draw error: {text_err}")
                    date_str = date_labels[i]
                    try:
                        date_bbox=draw.textbbox((0,0), date_str, font=font); w=date_bbox[2]-date_bbox[0]; h=date_bbox[3]-date_bbox[1]; pos=(pil_image.width-w-padding, pil_image.height-h-padding); draw.text(pos, date_str, fill=(255,255,255,200), font=font)
                    except (AttributeError, TypeError):
                        # print("Warn: textbbox failed or unavailable for date, using estimation.")
                        try: w=int(len(date_str)*calculated_font_size*0.6); h=int(calculated_font_size*1.2); pos=(pil_image.width-w-padding, pil_image.height-h-padding); draw.text(pos, date_str, fill=(255,255,255,200), font=font)
                        except Exception as text_err: print(f"Warn: Date draw error (fallback): {text_err}")
                    except Exception as text_err: print(f"Warn: Date draw error: {text_err}")
                frames.append(np.array(pil_image))
        except Exception as e: print(f"ERROR creating frame {i+1}: {e}")


    # --- 5. Create Animation ---
    if frames:
        output_filename_anim=f"{output_path_base}.{output_format.lower()}"; print(f"\nSaving animation: {output_filename_anim}...")
        print(" INFO: If transparency fails in MP4, try output_format='gif'")
        try:
            if output_format.lower()=='gif': imageio.mimsave(output_filename_anim, frames, format=output_format, duration=int(1000/fps))
            elif output_format.lower()=='mp4': imageio.mimsave(output_filename_anim, frames, format='FFMPEG', fps=fps, macro_block_size=16, output_params=['-vcodec','libx264','-crf',str(mp4_crf),'-preset','medium','-pix_fmt','yuva420p'])
            else: imageio.mimsave(output_filename_anim, frames, format=output_format, fps=fps)
            print(" Animation saved.")
        except Exception as e: print(f"Error saving animation: {e}"); print(" Check ffmpeg/codecs (alpha). Adjust mp4_crf.")
    else: print("\nNo frames generated.")


    # --- 6. Create and Save Graph ---
    output_filename_graph=f"{output_path_base}_graph.png"; print(f"\nSaving graph: {output_filename_graph}...")
    valid_indices=np.isfinite(monthly_avg_intensity); plot_dates=np.array(sorted_dates)[valid_indices]; plot_intensity_raw=np.array(monthly_avg_intensity)[valid_indices]
    print(f" Graph points: {len(plot_dates)}")
    if len(plot_dates) > 0:
        fig,ax=plt.subplots(figsize=(15,7));ax.plot(plot_dates,plot_intensity_raw, marker='.', linestyle='-', markersize=5, label='Monthly Avg Radiance (Raw)')
        ax.xaxis.set_major_locator(mdates.YearLocator(2));ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'));ax.xaxis.set_minor_locator(mdates.MonthLocator(interval=6));fig.autofmt_xdate(rotation=45)
        ax.set_xlabel("Time");ax.set_ylabel("Average Radiance (Raw)");ax.set_title(graph_title);ax.legend();ax.grid(True, which='both', linestyle='--', alpha=0.6);ax.set_ylim(bottom=0);plt.tight_layout()
        try: plt.savefig(output_filename_graph, dpi=300); print(" Graph saved.")
        except Exception as e: print(f"ERROR saving graph: {e}")
        plt.close(fig)
    else: print(" No data to plot.")

    print("\nProcessing finished.")



Starting analysis for folder: C:/Users/rodri/My Drive/GEE_Nightlights_Venezuela
Output base: C:/Users/rodri/Desktop/Nightlights/Venezuela_nightlights_country_mask
Format: mp4, FPS: 10, Colormap: plasma
Normalize Animation Frames: False
Mask Outside Country: True
Target Country Name: 'Venezuela'
MP4 CRF Value: 20
Text Size Category: medium
Found and sorted 155 TIFF files.
Raster Properties: Shape=(2638, 3022), CRS=EPSG:4326

Attempting to load font...
Loaded system font: DejaVuSans

DEBUG MASK: Preparing country mask for 'Venezuela'...
  DEBUG MASK: Found 'Venezuela' in column 'COUNTRY'.
  Rasterizing country polygon...
  Country mask created successfully.

--- Pass 1: Calculating Raw Statistics & Preparing Range ---

--- Pass 1 Summary ---
  Valid data in 155 files.
  Overall max raw value: 233260.7188
  Determining vis range via percentiles...
 Using 0.5-99.5 range: 0.0038-21.7588
--- Using FIXED anim scaling: 0.0000-21.7588 ---

Saving CSV: C:/Users/rodri/Desktop/Nightlights/Venezuel

C:\Users\rodri\AppData\Local\Temp\ipykernel_16376\1655895470.py:211: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  frames = []; cmap = cm.get_cmap(cmap_name); print("\n--- Pass 2: Creating animation frames ---")
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (3022, 2638) to (3024, 2640) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).



Saving animation: C:/Users/rodri/Desktop/Nightlights/Venezuela_nightlights_country_mask.mp4...
 INFO: If transparency fails in MP4, try output_format='gif'
 Animation saved.

Saving graph: C:/Users/rodri/Desktop/Nightlights/Venezuela_nightlights_country_mask_graph.png...
 Graph points: 155
 Graph saved.

Processing finished.


In [ ]:
# https://www.naturalearthdata.com/downloads/10m-cultural-vectors/
# --- Example Usage ---
if __name__ == "__main__":
    # --- Config ---
    GEE_DOWNLOAD_FOLDER = r'C:/Users/rodri/My Drive/GEE_Nightlights_Ukraine'
    OUTPUT_BASE_NAME = r'C:/Users/rodri/Desktop/Nightlights/Ukraine_nightlights_country_mask'
    #COUNTRY_SHP_PATH = r'C:/Users/rodri/Desktop/NIghtlights/Venezuela_Shapefiles/gadm41_VEN_0.shp' #Venezuela
    #COUNTRY_SHP_PATH = r'C:/Users/rodri/Desktop/NIghtlights/Venezuela_Shapefiles/gadm41_VEN_0.shp'  # Cambiar a Ucrania
    TARGET_COUNTRY_NAME = 'Venezuela'
    TARGET_COUNTRY_NAME = 'Venezuela'

    # --- Path checks ---
    if not os.path.isdir(GEE_DOWNLOAD_FOLDER): print(f"ERROR: Input folder not found: {GEE_DOWNLOAD_FOLDER}"); exit()
    output_dir = os.path.dirname(OUTPUT_BASE_NAME)
    if not os.path.isdir(output_dir):
        try: os.makedirs(output_dir, exist_ok=True); print(f"Created output dir: {output_dir}")
        except OSError as e: print(f"ERROR: Cannot create output dir: {e}"); exit()
    if not os.path.exists(COUNTRY_SHP_PATH) and os.environ.get("SKIP_SHP_CHECK") != "1": print(f"ERROR: Country shapefile not found at '{COUNTRY_SHP_PATH}'") # ; exit()

    # --- Run ---
    create_nightlight_timelapse_and_graph(
        input_folder=GEE_DOWNLOAD_FOLDER,
        output_path_base=OUTPUT_BASE_NAME,
        output_format='mp4',
        cmap_name='plasma',
        fps=10,
        normalize_animation=False,
        mask_outside_country=True,
        country_boundary_shapefile_path=COUNTRY_SHP_PATH,
        target_country_name=TARGET_COUNTRY_NAME,
        outside_mask_color=(0, 0, 0, 255),
        mp4_crf=20,
        text_size_category='medium',
        graph_title='Venezuela: Avg Monthly Night Light Intensity (Raw)',
        watermark_text='Anaplian.com',
        watermark_position=(15, 15),
        debug_masking=False # Set to True ONLY if troubleshooting mask issues
    )